<a href="https://colab.research.google.com/github/saided/pandas_using_python/blob/master/GRP_Creation_final_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import requests
import xlrd
import openpyxl
from collections import ChainMap
from requests import Timeout

In [4]:
def root_group():
    url = "https://api.cloud.jiosurveillance.com/auth/oauth/token"
    username = input("Enter account ID")
    password = input("Enter account password")

    payload = 'client_secret=8b078e3496893ae3b9d7ccb83a6befdb&client_id=firmware-upgrade-job&grant_type=password&username={}&password={}'.format(
    username, password)
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
            }
    try:
        response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
        #print(response.text.encode('utf8'))
        #print(response.json())
    except: Timeout

    r = response.json()
    access_token = r['access_token']
    token = str(access_token)
    #print(token)
    own_id = r['owner_id']

    url = "https://api.cloud.jiosurveillance.com/folders?op=FIND"

    payload = "{\n\t\"user\" : \"%s\"\n}" % (own_id)
    headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % (token)
    }
    response = requests.request("POST", url, headers=headers, data=payload, timeout=6)

    own_response = response.json()
    print(own_response)
    b = own_response['result']
    c = b['items']
    #print(c[0])
    #print(len(c))
    #print('Total no.of existing groups in the account :',len(c)-1)
    mast_grp_id = c[0]['id']
    #print(mast_grp_id,type(mast_grp_id))
    if len(c)==0:
      print('something went wrong')
    return(mast_grp_id,own_id,token)

print(root_group())  

Enter account IDrr.test4@gmail.com
Enter account passwordJioadmin@123
{'success': True, 'result': {'items': [{'id': '801-Z6hTHMk9Ii', 'name': '__root__', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': True, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}]}}
('801-Z6hTHMk9Ii', 801000012219, '801-Ub8289477-c584-4a48-8d18-878c6ee72445')


In [47]:
is_root_grp = list(root_group())
print(len(is_root_grp))
mast_grp_id = is_root_grp[0]
own_id = is_root_grp[1]
token = is_root_grp[2]
#print(token,type(token))

Enter account IDrr.test4@gmail.com
Enter account passwordJioadmin@123
{'success': True, 'result': {'items': [{'id': '801-Z6hTHMk9Ii', 'name': '__root__', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': True, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}]}}
3


In [48]:

def create_group(cellvalue,grp_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + grp_id + '\",\n\t\"name\" : \"' + cellvalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)




In [ ]:
North=0
theFile = openpyxl.load_workbook('/content/Tree structure.xlsx')
allSheetNames = theFile.sheetnames
#print(allSheetNames)
print("All sheet names {} " .format(allSheetNames))
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  print('Total no of rows',sh.nrows)
  print('Total no of columns',sh.ncols)
  zone = []
  zone_folder = []
  northList = []
  northtownList = []
  jplist = []
  iplist = []
  state_folder = []
  town_folder = []
  jp_folder = []
  ip_folder = []
  zone_id = []
  state_id = []
  town_id = []
  jp_id = []
  ip_id = []
  i= 0
  j=0
  k = 0
  l=0
  m = 0
  for row in range(1, sh.nrows):
    for col in range(sh.ncols):
      mycell = sh.cell(row,col).value
      if (len(is_root_grp) !=0 and col==6):
        if mycell not in zone:
             zone.append(mycell)
             print(zone)
             zone_folder.append(create_group(mycell,mast_grp_id,token))
             zone_id.append(zone_folder[i]['result']['id'])
             i+=1
        else:
           break
    
    for col in range(1, 7):
      mycell = sh.cell(row,col).value
      for x in range(len(zone)):
          if mycell==zone[x]:
            print(x,mycell)
            print(row,col)
            statecell = sh.cell(row, col+1).value
        #statecell.value = str(statecell.value)
            if statecell.upper() in [name.upper() for name in northList]:
              break
            else:
              northList.append(statecell)
              print("NorthList contains following list" + str(northList))
              state_folder.append(create_group(statecell,zone_id[x],token))
              print(j)
              state_id.append(state_folder[j]['result']['id'])
              j+=1
    for col in range(0, 9):
      mycell = sh.cell(row,col).value
      for x in range(len(northList)):
          if mycell == northList[x]:
            town = sh.cell(row, col+1).value
            if town.lower() in [name.lower() for name in northtownList]:
              break
            else:
              northtownList.append(town)
              town_folder.append(create_group(town,state_id[x],token))
              print(k)
              town_id.append(town_folder[k]['result']['id'])
              k+=1
    for col in range(1, 11):
      mycell = sh.cell(row,col).value
      for x in range(len(northtownList)):
          if mycell ==northtownList[x] :
            jp = sh.cell(row,col+2).value
            if jp.lower() in [name.lower() for name in jplist]:
              break
            else:
              jplist.append(jp)
              jp_folder.append(create_group(jp,town_id[x],token))
              jp_id.append(jp_folder[l]['result']['id'])
              l+=1
   
    for col in range(1, 11):
      mycell = sh.cell(row,col).value
      for x in range(len(jplist)):
          if mycell ==jplist[x] :
            ip = sh.cell(row,col-9).value
            if ip.lower() in [name.lower() for name in iplist]:
              break
            else:
              iplist.append(ip)
              ip_folder.append(create_group(ip,jp_id[x],token))
              ip_id.append(ip_folder[m]['result']['id'])
              m+=1
            

    
    
    



In [32]:
#print(zone_folder)
#zone_id = zone_folder[0]['result']['id']
print(zone_id)
print(state_folder[2])
print(northList)
print(len(state_id))
print(len(town_id))

['801-JYoAiuXOQO', '801-Kma9oZDKvW', '801-XyeXGF2Ou9', '801-pngyzrsLoX']
{'success': True, 'result': {'id': '801-QAX2zBBKBD', 'name': 'MADHYA PRADESH', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}
['MAHARASHTRA', 'ORISSA', 'MADHYA PRADESH', 'GUJARAT', 'JAMMU KASHMIR', 'TAMIL NADU', 'ASSAM', 'HARYANA', 'RAJASTHAN', 'UP WEST', 'UTTARAKHAND', 'ANDHRA PRADESH', 'KERALA', 'UP EAST1', 'JHARKHAND']
15
62


In [42]:
print(town_folder[26])
print(jplist)

{'success': True, 'result': {'id': '801-XVYCibkPPz', 'name': 'Bhadrak', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}
['I-MH-KUDL-ENB-9003', 'I-OR-CNDK-ENB-6000', 'I-OR-BETI-ENB-6004', 'I-OR-BLSR-ENB-9005', 'I-OR-RRKL-ENB-6010', 'I-OR-KTPD-ENB-9002', 'I-MP-JBLP-ENB-0002', 'I-GJ-BVLA-ENB-9018', 'I-OR-RRKL-ENB-6015', 'I-OR-RRKL-ENB-6032', 'I-OR-BRRI-ENB-9001', 'I-OR-CNDP-ENB-6002', 'I-OR-JGTP-ENB-6003', 'I-OR-PRDP-ENB-6002', 'I-OR-CCKK-ENB-6030', 'I-OR-GOLT-ENB-9004', 'I-JK-RESI-ENB-0005', 'I-TN-LLGD-ENB-9009', 'I-MP-SHOR-ENB-9011', 'I-AS-RMKS-ENB-9011', 'I-OR-RRKL-ENB-6034', 'I-MP-RTLM-AG1-4400', 'I-MP-GUNA-ENB-9010', 'I-AS-TNSK-ENB-6002', 'I-OR-SNTL-ENB-9005', 'I-OR-RGNP-ENB-9021', 'I-HR-GHAN-ENB-9019', 'I-RJ-PNDW-ENB-9015', 'I-OR-KNDP-ENB-9003', 'I-OR-RGNP-ENB-6005', 'I-OR-SUKN-ENB-9004', 'I-RJ-PNDW-ENB-